In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.interpolate import interp1d
import numpy as np
import scipy as sp
import torch as tn
import random
import warnings
import multiprocessing as mp
from functools import partial
from scipy.integrate import quad, IntegrationWarning
import time


# Import files
from constants import *
from functions import sigma_sm, sme
from rotation import *

In [ ]:
# Quarks
quarks = [
    (2, 2/3*e, 'u', 1/2),
     # (1, -1/3*e, 'd', -1/2),
     # (3, -1/3*e, 's', -1/2),
     # (4, 2/3*e, 'c', 1/2),
     #  (5, -1/3*e, 'b', -1/2),
     # (6, 2/3*e, 't', 1/2),
]

# List of quark properties and couplings
quark_couplings = []

for flavor, e_f, name, I3 in quarks:
    g_fR = -e_f * sin2th_w
    g_fL = I3 - e_f * sin2th_w
    
    # Rounding to 4 decimal places
    e_f = round(e_f, 10)
    g_fR = round(g_fR, 10)
    g_fL = round(g_fL, 10)
    
    quark_couplings.append((flavor, e_f, g_fR, g_fL))

print(quark_couplings)

In [ ]:
g = tn.tensor([
    [1,0,0,0],
    [0,-1,0,0],
    [0,0,-1,0],
    [0,0,0,-1]
], dtype=tn.float32)
CR = tn.tensor([
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
], dtype=tn.float32)

In [ ]:
# Define the constant tensors once
p1 =  0.5*tn.tensor([1, 0, 0, 1], dtype=tn.float32)
p2 =  0.5*tn.tensor([1, 0, 0, -1], dtype=tn.float32)

# Precompute total number of steps

specific_time = datetime(2017, 1, 1, 0, 0)

start_time = int(specific_time.timestamp())

# start_time = int(time.time())
end_time = start_time + int(timedelta(days=1).total_seconds())
step_seconds = int(timedelta(hours=1).total_seconds())
num_steps = (end_time - start_time) // step_seconds

# Lists to store the times and contr matrix elements
times = []
contrelep1 = []
contrelep2 = []

R_y_lat = R_y(latitude)
R_z_azi = R_z(azimuth)
mat_cons = tn.matmul(R_y_lat,R_z_azi)
# Main loop
current_time = start_time
for _ in range(num_steps):
    # Convert current_time to a timestamp
    current_datetime = datetime.fromtimestamp(current_time)
    time_utc = current_datetime.timestamp()

    # Calculate omega_t
    omega_t_sid = omega_utc * time_utc + 3.2830 
    # Construct the complete rotation matrix from SCF to CMS
    R_Z_omega = R_Z(omega_t_sid)
    R_mat = tn.matmul(R_Z_omega, mat_cons)
    R_matrix1 = tn.einsum('ma,an->mn', g, R_mat)
    R_matrix2 = tn.einsum('am,na->mn', g, R_mat)
    # print(R_matrix1)
    # Compute contrL and contrR using matrix multiplication
    contrp1 = tn.einsum('ij,j->i', R_matrix1, p1)
    contrp2 =  tn.einsum('ij,i->j',R_matrix2, p2)
    # Record the times and contr matrix elements
    times.append(current_time)
    contrelep1.append(contrp1)
    contrelep2.append(contrp2)


    # Move to the next time step
    current_time += step_seconds

# Function to convert timestamps to hours
def convto_hours(timestamps):
    start_time = timestamps[0]  # The start time to normalize
    return [(t - start_time) / 3600 for t in timestamps]  # Convert seconds to hours

# Perform conversion
hours_start = convto_hours(times)

In [ ]:
# Create the linear values
linear_values = tn.linspace(-1.75e-5, 1.75e-5, steps=20)

# Initialize an empty list to hold the tensors
CL_values = []

# Loop through the linear values and create the tensors
for value in linear_values:
    tensor = tn.zeros((4, 4), dtype=tn.float32)  # Create a tensor of the specified size, filled with zeros
    tensor[1, 2] = value  # Set the value at the first index
    tensor[2, 1] =  value  # Set the value at the second index
    CL_values.append(tensor)

for i in range(20):
    locals()[f"CL{i+1}"] = CL_values[i]

In [ ]:
%%time
# Calculate sigma_sm
Q_min = 70
Q_max = 80
sigma_sm_value = sigma_sm(Q_min, Q_max, quark_couplings)

In [ ]:
warnings.simplefilter("ignore", IntegrationWarning)

# Define the class at the top level, so it can be pickled
class SMECrossSection:
    def __init__(self, pm, pn, quark_couplings, CL_values, CR, sigma_sm_value, Q_min, Q_max):
        self.pm = pm
        self.pn = pn
        self.quark_couplings = quark_couplings
        self.CL_values = CL_values
        self.CR = CR
        self.sigma_sm_value = sigma_sm_value
        self.Q_min = Q_min
        self.Q_max = Q_max

    def compute_sme(self, CL):
        return sme(self.Q_min, self.Q_max, CL, self.CR, self.pm, self.pn, self.quark_couplings, self.sigma_sm_value)

    def compute_all_results(self):
        results = {}
        for i, CL in enumerate(self.CL_values, 1):
            sme_result = self.compute_sme(CL)
            final_result = sme_result + self.sigma_sm_value
            results[f'result_sme{i}'] = final_result
        return results

# This function is now at the top level, making it pickleable
def compute_wrapper(args):
    pm, pn, quark_couplings, CL_values, CR, sigma_sm_value, Q_min, Q_max = args
    sme_calculator = SMECrossSection(pm, pn, quark_couplings, CL_values, CR, sigma_sm_value, Q_min, Q_max)
    return sme_calculator.compute_all_results()

# Main computation function
def run_computation(contrelep1, contrelep2, quark_couplings, CL_values, CR, sigma_sm_value, Q_min, Q_max):
    warnings.simplefilter("ignore", IntegrationWarning)
    
    # Prepare the arguments for the computation
    args_list = [(pm, pn, quark_couplings, CL_values, CR, sigma_sm_value, Q_min, Q_max)
                 for pm, pn in zip(contrelep1, contrelep2)]
    
    # Create a multiprocessing pool and run the computation in parallel
    with mp.Pool(mp.cpu_count()) as pool:
        results = pool.map(compute_wrapper, args_list)
    
    return results

In [ ]:
%%time

results = run_computation(contrelep1, contrelep2, quark_couplings, CL_values, CR, sigma_sm_value, Q_min, Q_max)
sme_array = [np.array([result[f'result_sme{i+1}']  for result in results]) for i in range(20)]
sme_array_plot = [np.array([result[f'result_sme{i+1}']/ sigma_sm_value for result in results]) for i in range(20)]
hours_array = np.array(hours_start)

In [ ]:
plt.figure(figsize=(10, 8))

# Use the updated method to get a colormap with 20 distinct colors
colors = plt.colormaps['tab20'](np.linspace(0, 1, 20))

# Plot each dratio
for i in range(20):
    plt.step(hours_array, sme_array_plot[i], where='post', color=colors[i], 
             label=f'$C_{{L,{i+1}}}$', linewidth=1.5)

# Customizing the legend
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=10, frameon=True)

# Adding labels and title 
plt.xlabel('Time (hours)', fontsize=12)
plt.ylabel('$\\sigma_{LIV} \;contribution$', fontsize=12)
plt.title('$ Q \in [70,80] \;GeV$', fontsize=16, loc='left')
# plt.text(12, 0.39, '2018', fontsize=20, horizontalalignment='center')

plt.minorticks_on()
plt.tick_params(axis='x', which='minor', bottom=False)  
plt.tick_params(which='both', width=1)
plt.tick_params(which='major', length=7)
plt.tick_params(which='minor', length=4, color='gray')
plt.tick_params(axis='y', direction='in', which ='both') 

plt.xticks(ticks=range(0, 24, 1), labels=[str(hour) for hour in range(0, 24, 1)])

# Adjust margins manually
plt.subplots_adjust(left=0.1, right=0.75, top=0.9, bottom=0.15)

# Save the plot
# plt.savefig("liv_all.png", bbox_inches='tight', pad_inches=0.1)
plt.show()

# $\chi^2$ Fit
## $$\chi^2 = \frac{1}{\sigma^2_{st}} \sum_i \left(\sigma_{SM} - \sigma_{SME,i}\right)^2$$


In [ ]:
chi2_values_dict = {}
obs = sigma_sm_value  

for key in results[0].keys():
    chi2 = np.sum([( obs - result[key])**2 / (0.07)**2 for result in results])
    chi2_values_dict[key] = chi2


In [ ]:
# Assuming `linear_values` and `chi2_values_dict` are already defined
plt.figure(figsize=(8, 6))

# Extract x and y values from the dictionary
x_values = np.array(linear_values)
y_values = np.array(list(chi2_values_dict.values()))

# Plot the chi-square values with markers and a line
plt.plot(x_values, y_values, marker='o', linestyle='-', color='blue', markersize=6, linewidth=2)

# Plot horizontal line at y=1
plt.axhline(y=1, color='red', linestyle='--', linewidth=1.5)

# Interpolate to find more precise intersection points
interpolation_func = interp1d(x_values, y_values - 1, kind='linear')
# Find where the interpolated curve crosses y=1
roots = np.where(np.diff(np.sign(y_values - 1)))[0]

# Calculate exact x values for intersections and plot partial vertical lines
for root in roots:
    x_intersection = x_values[root] - interpolation_func(x_values[root]) * (x_values[root + 1] - x_values[root]) / (y_values[root + 1] - y_values[root])
    
    # Calculate the y-position of the intersection point
    y_intersection = 1

    # Plot a vertical line from the intersection point down to the x-axis
    plt.vlines(x_intersection, ymin=0, ymax=y_intersection, color='green', linestyle='--', linewidth=1.5)
    
    # Mark the intersection point
    plt.plot(x_intersection, y_intersection, 'ro')

# Set x-axis label and format the x-axis values to be in the form of 1, 2, etc.
ax = plt.gca()
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=True, useMathText=True))
ax.xaxis.get_major_formatter().set_scientific(False)
ax.xaxis.get_major_formatter().set_useOffset(True)

# Automatically adjust the offset notation to show 10^-5 on the x-axis
ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))

# Label axes with units if applicable
plt.xlabel('C', fontsize=14)
plt.ylabel('$\\chi^2$', fontsize=14)
plt.title('Sensitivity of Lorentz Violation to Wilson Coefficients in SME: $\\chi^2$ Analysis', fontsize=12)

# Ensure the y-axis has integer values
ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

# Add grid lines for both axes
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Use a tight layout to avoid clipping
plt.tight_layout()
plt.savefig('likelihood.png')
# Show the plot
plt.show()
